爬取華視新聞網的新聞
* 利用本身網址即可
>1. [政治](https://news.cts.com.tw/politics/index.html) 
>2. [國際](https://news.cts.com.tw/international/index.html)
>3. [財經](https://news.cts.com.tw/money/index.html)
>4. [社會](https://news.cts.com.tw/society/index.html)
>5. [地方](https://news.cts.com.tw/local/index.html)
>6. [綜合](https://news.cts.com.tw/general/index.html)





In [ ]:
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 財經 https://news.cts.com.tw/money/index.html

In [ ]:
url = "https://news.cts.com.tw/money/index.html"
response = requests.get(url, verify = False)
response.encoding = "utf-8" # 解決中文亂碼問題
html = BeautifulSoup(response.text)
# html


| Title     | Description  |
| ----------|:-------------:|
| Label     | Real = 1, Fake = 0
| Headline  | Headline of News            |
| Link      | url of News |
| Date      | Published Date      |  
| Source    | Source of the news. (One who can verify the claim of the news)| 
| Category  | Category of News            |
| Content   | Article or body of the news |

In [ ]:
# 建立pandas表格
table = {
    "Headline" : [],
    "Link" : [],
    "Date" : [],
    "Source" : [],
    "Category" : [],
    "Content" : []
}

In [ ]:
newslists = html.find("div", class_ = "newslist-container flexbox").find_all("a")
# print(newslists)

t = 0
# 取得新聞標題、連結
for newslist in newslists:
    title = newslist["title"]
    link = newslist["href"]
    date = newslist.find("span", class_ = "newstime").text
    # print(date)
    # 放入table
    table["Headline"].append(title)
    table["Link"].append(link)
    table["Date"].append(date)
    t = t + 1
print(t)

200


In [ ]:
# 解決"InsecureRequestWarning"
import urllib3
urllib3.disable_warnings()
from urllib3 import PoolManager
http = PoolManager()

單篇文章試做:

```
url_ = 'https://news.cts.com.tw/cts/politics/202105/202105262043804.html'
response_ = requests.get(url_, verify = False)
response_.encoding = "utf-8"
html_ = BeautifulSoup(response_.text)
content_ = html_.find("div", class_ = "artical-content")
print(type(content_))
t = content_.find_all("p")
for line in t:
    print(line.text)
```

In [ ]:
# 將各文章連結讀取成html

ti = 0
for newslist in newslists:
    link = newslist["href"]
    print(link)

    # 抓出新聞種類，利用網址split方式取得
    category = link.split("/")[-3]
    table["Category"].append(category)

    # http.request('GET', link, headers={'Accept-Encoding': 'br'})
    n_response = requests.get(link, verify = False)
    n_response.encoding = "utf-8"
    n_html = BeautifulSoup(n_response.text)

    # 抓出Date
    # date = n_html.find("time", class_ = "artical-time").text
    # table["Date"].append(date)
    # print(date)

    # 抓出Content
    contents = n_html.find("div", class_ = "artical-content")
    ti = ti + 1
    # 直接利用p視為第1段...第n段，再使用for-in依序取得內文
    # 再將每個p以list格式合併儲存，而後再使用join使其裝在一起
    li = []
    t = contents.find_all("p")
    for line in t:
        content = line.text.strip()
        li.append(content)
        joinli = ",".join(li)
    # print(joinli)
    table["Content"].append(joinli)
    # print("=" * 10)

    # 抓出資料來源(Source)
    source = t[-1].text.strip().split("：")[-1]
    table["Source"].append(source)

# print(ti) # 檢查資料有幾筆

https://news.cts.com.tw/cts/money/202106/202106172046367.html
https://news.cts.com.tw/gvm/money/202106/202106152046181.html
https://news.cts.com.tw/cts/money/202106/202106152046105.html
https://news.cts.com.tw/cts/money/202106/202106132045909.html
https://news.cts.com.tw/cts/money/202106/202106122045856.html
https://news.cts.com.tw/cts/money/202106/202106112045693.html
https://news.cts.com.tw/cts/money/202106/202106102045552.html
https://news.cts.com.tw/cts/money/202106/202106092045506.html
https://news.cts.com.tw/cts/money/202106/202106092045420.html
https://news.cts.com.tw/cts/money/202106/202106082045369.html
https://news.cts.com.tw/cts/money/202106/202106082045317.html
https://news.cts.com.tw/cts/money/202106/202106082045274.html
https://news.cts.com.tw/cna/money/202106/202106072045170.html
https://news.cts.com.tw/cts/money/202106/202106062045068.html
https://news.cts.com.tw/cts/money/202106/202106042044843.html
https://news.cts.com.tw/cts/money/202106/202106032044698.html
https://

In [ ]:
df = pd.DataFrame(table)
df

,Headline,Link,Date,Source,Category,Content
0,2021/06/17 國際財經最前線 歐美股市指數,https://news.cts.com.tw/cts/money/202106/20210...,2021/06/17 09:04,華視新聞,money,"綜合報導 / 台北市,來關心今天最新的歐美股市指數及金價油價，美股最$線，道瓊工業3403..."
1,遠見／ 證券業〉推廣金融教育，外加設計美學！大慶證改造「老號子」教菜籃族懂理財,https://news.cts.com.tw/gvm/money/202106/20210...,2021/06/15 15:30,遠見,money,"文．許雅綿 / 台北市,92歲的老奶奶一早帶便當出門，來到俗稱「號子」的券商營業廳，直到下..."
2,幫你讀懂全世界 丁解析G7最低稅率,https://news.cts.com.tw/cts/money/202106/20210...,2021/06/15 09:23,華視新聞,money,"綜合報導 / 台北市,財經專家丁學文，今天要為我們導讀的，是《倫敦金融時報》最新的一篇全球..."
3,從油價看通膨 丁學文幫你讀懂全世界,https://news.cts.com.tw/cts/money/202106/20210...,2021/06/13 11:22,華視新聞,money,"綜合報導 / 台北市,台灣的疫情還在延燒，但國際金融市場，已經明顯往「疫後」邁進。經濟復甦..."
4,全球經濟大預測 丁學文幫你讀懂全世界,https://news.cts.com.tw/cts/money/202106/20210...,2021/06/12 08:48,華視新聞,money,"綜合報導 / 台北市,台灣人民正在和疫情搏鬥，不過歐美許多國家的疫情，已經慢慢平息。特別是..."
...,...,...,...,...,...,...
195,幫你讀懂全世界 丁學文批注經濟學人,https://news.cts.com.tw/cts/money/202103/20210...,2021/03/17 10:43,華視新聞,money,"綜合報導 / 台北市,新冠疫情重創全球經濟，許多國家都猛印鈔票應急。但這樣的措施，卻有隱憂..."
196,2021/03/17 國際財經最前線 歐美股市指數,https://news.cts.com.tw/cts/money/202103/20210...,2021/03/17 08:34,華視新聞,money,"綜合報導 / 台北市,來關心今(17)日最新的歐美股市指數及金價油價。美股最$線，道瓊工業..."
197,2021/03/16台積電翻紅收高 台股漲63點收16300點,https://news.cts.com.tw/cts/money/202103/20210...,2021/03/16 15:19,華視新聞,money,"綜合報導 / 台北市,台股今在IC設計等電子股領軍下，指數緩步墊高，尾盤在台積電大單敲進，..."
198,2021/03/16 國際財經最前線 歐美股市指數,https://news.cts.com.tw/cts/money/202103/20210...,2021/03/16 08:40,華視新聞,money,"綜合報導 / 台北市,來關心今(16)日最新的歐美股市指數及金價油價，美股最$線，道瓊工業..."


In [ ]:
# 存成csv檔

path = "drive/Shareddrives/Fake_news_detecter/datasets/CTS"
df.to_csv(path + "/" + category + ".csv", encoding = "utf-8", index = False)